---
layout: base
title: Hallpass
permalink: /bathroom/hallpass
menu: nav/toolkits/bathroom/menu.html
toc: false
---

{% include nav/toolkits/bathroom/menu.html %}

<div class="container">
    <div class="components">
        <!-- Resized Image -->
        <img src="https://github.com/user-attachments/assets/c350ab11-f560-4c76-b1f3-8bfbce017f77" alt="Teacher" style="max-width: 300px; height: auto; display: block; margin: 20px auto;">

        <p>Good job not using any paper, <span id="studentNameDisplay"></span>! You're saving both the environment and your seed!</p>

        <button type="button" onclick="window.approveStudent()">I'm done</button>
    </div>
</div>

<script type="module">
  import { javaURI, fetchOptions } from "{{site.baseurl}}/assets/js/api/config.js";

  const teacherName = "jm1021@gmail.com"; // Replace with dynamic teacher name if needed

  const url = javaURI + `/api/queue`;
  const approveUrl = url + '/approve';
  const adminUrl = `${javaURI}/api/tinkle/add`;
  const postOptions = {
      ...fetchOptions,
      method: 'POST',
  };

  window.addEventListener('load', () => {
    fetchUser();
  });

  async function fetchUser() {
      const response = await fetch(javaURI + `/api/person/get`, {
          method: 'GET',
          cache: "no-cache",
          credentials: 'include',
          headers: { 
              'Content-Type': 'application/json',
              'X-Origin': 'client' 
          }
      });
      if (response.ok) {
          const userInfo = await response.json();
          window.studentName = userInfo.name;
          document.getElementById("studentNameDisplay").textContent = window.studentName;
          console.log("Person: ", userInfo);
      }
  }

  // Function to approve student who is first in line
  window.approveStudent = async function() {
      const now = new Date();
      // Get current time formatted as HH:MM:SS
      const hours = now.getHours();
      const minutes = now.getMinutes().toString().padStart(2, "0");
      const seconds = now.getSeconds().toString().padStart(2, "0");
      const timeOut = `${hours}:${minutes}:${seconds}`;

      // Retrieve timeIn (which should be stored earlier)
      const timeIn = localStorage.getItem("timeIn");
      let timeSpentMinutes = 0;
      if (timeIn) {
          // Convert times from "HH:MM:SS" to total seconds
          const [inHours, inMinutes, inSeconds] = timeIn.split(":").map(Number);
          const [outHours, outMinutes, outSeconds] = timeOut.split(":").map(Number);
          const totalSecondsIn = inHours * 3600 + inMinutes * 60 + inSeconds;
          const totalSecondsOut = outHours * 3600 + outMinutes * 60 + outSeconds;
          const timeSpentSeconds = totalSecondsOut - totalSecondsIn;
          timeSpentMinutes = (timeSpentSeconds / 60).toFixed(2);
          console.log(`Time spent: ${timeSpentMinutes} minutes`);
      } else {
          console.log("No timeIn value stored in localStorage.");
      }

      // Create a period string using a double dash (you mentioned the format should be updated)
      const timeInOut = `${timeIn}-${timeOut}`;

      const times = {
          studentEmail: window.studentName,
          timeIn: timeInOut,
          averageDuration: timeSpentMinutes,
      };

      const queuer = {
          teacherEmail: teacherName,
          studentName: window.studentName,
          uri: javaURI,
      };

      try {
          // First, send data to adminUrl
          const adminResponse = await fetch(adminUrl, {
              ...postOptions,
              body: JSON.stringify(times),
          });
          if (adminResponse.ok) {
              console.log("Time added to database");
          } else {
              alert("Failed to add time to database.");
          }

          // After adminUrl fetch completes, send the approval request
          const approveResponse = await fetch(approveUrl, {
              ...postOptions,
              body: JSON.stringify(queuer),
          });
          if (approveResponse.ok) {
              console.log("Successful Approval");
          } else {
              alert("Failed to approve student.");
          }
      } catch (error) {
          console.error("Error with fetch requests:", error);
      }

      // Optionally, redirect the user after successful approval
      window.location.href = "{{site.baseurl}}/profile";
  }
</script>